In [1]:
%load_ext autoreload
%autoreload 2

# Available models
Available models, and model names are given here  <a href="https://msasg.visualstudio.com/QAS/_wiki/wikis/QAS.wiki/134728/Getting-Started-with-Substrate-LLM-API?anchor=available-models"> Substrate-LLM-API available models</a>.

In [2]:
import guidance

start to install package: redis
successfully installed package: redis
start to install package: redis-om


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.4.15 requires tokenizers>=0.13.2, but you have tokenizers 0.12.1 which is incompatible.
streamlit 0.82.0 requires click<8.0,>=7.0, but you have click 8.1.7 which is incompatible.


successfully installed package: redis-om


In [4]:
from guidance.llms._substrate_llm import SubstrateLLM

# AAD based authentication (preferred)

In [8]:

# # we will use GPT-3 for most of the examples in this tutorial
# guidance.llm = guidance.llms.OpenAI("text-davinci-003")

_ENDPOINT = """https://httpqas26-frontend-qasazap-prod-dsm02p.qas.binginternal.com/completions"
_SCOPES = ["api://68df66a4-cad9-4bfd-872b-c6ddde00d6b2/access"]
_CLIENT_ID = "68df66a4-cad9-4bfd-872b-c6ddde00d6b2"
_AUTHORITY = "https://login.microsoftonline.com/72f988bf-86f1-41af-91ab-2d7cd011db47"
guidance.llm = SubstrateLLM(model="gpt-3.5-turbo-cl100k_base",
                                          model_name="dev-gpt-35-turbo",
                                          caching=False,
                                          chat_mode=False, 
                                          client_id=_CLIENT_ID, 
                                          authority=_AUTHORITY, 
                                          endpoint=_ENDPOINT, 
                                          scopes=_SCOPES)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EMDN5RXDV to authenticate.


In [9]:
guidance.llm._tokenizer,  guidance.llm.chat_mode, guidance.llm.model_name, guidance.llm._rest_headers

(<Encoding 'cl100k_base'>,
 False,
 'dev-gpt-35-turbo',
 {'Content-Type': 'application/json',
  'X-ModelType': 'dev-gpt-35-turbo',
  'Accept': 'application/json'})

In [20]:
prompt ="""<|im_start|>system 
You are an NLP assistant whose purpose is to perform Named Entity Recognition (NER). NER involves identifying and classifying named entities in a text into predefined categories such as person names, organizations, locations, and others. You will need to use the tags defined below:
O means the word doesn’t correspond to any entity.
B-PER/I-PER means the word corresponds to the beginning of/is inside a person entity.
B-ORG/I-ORG means the word corresponds to the beginning of/is inside an organization entity.
B-LOC/I-LOC means the word corresponds to the beginning of/is inside a location entity.
Do not try to answer the question! Just tag each token in the sentence.<|im_end|>
<|im_start|>user
Tag the following sentence: \"Stratford , Oklahoma\"<|im_end|>
<|im_start|>assistant
Stratford_B-LOC ,_I-LOC Oklahoma_I-LOC<|im_end|>
<|im_start|>user
Tag the following sentence: \"Anders Behring Breivik\"<|im_end|>
<|im_start|>assistant
Anders_B-PER Behring_I-PER Breivik_I-PER<|im_end|>
<|im_start|>user
Tag the following sentence: \"'' The World as Best as I Remember It , Volume Two '' ( 1992 )\"<|im_end|>
<|im_start|>assistant
''_O The_B-ORG World_I-ORG as_I-ORG Best_I-ORG as_I-ORG I_I-ORG Remember_I-ORG It_I-ORG ,_I-ORG Volume_I-ORG Two_I-ORG ''_O (_O 1992_O )_O<|im_end|>
<|im_start|>user
Tag the following sentence: \"24 Apr – Heinrich Himmler\"<|im_end|>
<|im_start|>assistant
24_O Apr_O –_O Heinrich_B-PER Himmler_I-PER<|im_end|>
<|im_start|>user
Tag the following sentence: \"`` Flight of Icarus '' ( Tierra Santa )\"<|im_end|>
<|im_start|>assistant
``_O Flight_B-ORG of_I-ORG Icarus_I-ORG ''_O (_O Tierra_B-ORG Santa_I-ORG )_O<|im_end|>
<|im_start|>user
Tag the following sentence: \"Bath and North East Somerset\"<|im_end|>
<|im_start|>assistant
Bath_B-LOC and_I-LOC North_I-LOC East_I-LOC Somerset_I-LOC<|im_end|>
<|im_start|>user
Tag the following sentence: \"Communes of the Marne department\"<|im_end|>
<|im_start|>assistant
Communes_B-LOC of_I-LOC the_I-LOC Marne_I-LOC department_I-LOC<|im_end|>
<|im_start|>user
Tag the following sentence: \"1970 — did not compete\"<|im_end|>
<|im_start|>assistant
1970_B-ORG —_O did_O not_O compete_O<|im_end|>
<|im_start|>user
Tag the following sentence: \"' '' Bruno Ruffo '' '\"
<|im_end|>
<|im_start|>assistant"""

In [15]:
from guidance.llms._openai import prompt_to_messages
from guidance.llms._substrate_llm import messages_to_guidance, messages_to_prompt

In [21]:
messages = prompt_to_messages(prompt)

In [23]:
messages

[{'role': 'system',
  'content': 'You are an NLP assistant whose purpose is to perform Named Entity Recognition (NER). NER involves identifying and classifying named entities in a text into predefined categories such as person names, organizations, locations, and others. You will need to use the tags defined below:\nO means the word doesn’t correspond to any entity.\nB-PER/I-PER means the word corresponds to the beginning of/is inside a person entity.\nB-ORG/I-ORG means the word corresponds to the beginning of/is inside an organization entity.\nB-LOC/I-LOC means the word corresponds to the beginning of/is inside a location entity.\nDo not try to answer the question! Just tag each token in the sentence.'},
 {'role': 'user',
  'content': 'Tag the following sentence: "Stratford , Oklahoma"'},
 {'role': 'assistant', 'content': 'Stratford_B-LOC ,_I-LOC Oklahoma_I-LOC'},
 {'role': 'user',
  'content': 'Tag the following sentence: "Anders Behring Breivik"'},
 {'role': 'assistant', 'content': 

In [31]:
import tiktoken
substrate_output= ["\'_\'", 'Bruno_B-PER', 'Ruffo_I-PER', "\'\'_O", "\'_O"],
expected_output= ["\'_O", "\'\'_O", 'Bruno_B-PER', 'Ruffo_I-PER', "\'\'_O", "\'_O"]

# strip whitespace from each item in the substrate_output and expected_output lists
for i in substrate_output:
    print(i)
    break
# substrate_output = [item.strip() for item in substrate_output]

["'_'", 'Bruno_B-PER', 'Ruffo_I-PER', "''_O", "'_O"]


In [ ]:
list(map(lambda x: x.strip(), substrate_output))

# substrate_output = list(map(lambda x: x.strip(), substrate_output))
# expected_output = list(map(lambda x: x.strip(), expected_output))
# substrate_output = " ".join(substrate_output)
# expected_output = " ".join(expected_output)

# enc = tiktoken.get_encoding("cl100k_base")
# encoded_substrate_output = enc.encode(substrate_output)
# encoded_expected_output = enc.encode(expected_output)

# assert enc.decode(enc.encode("hello world")) == "'''BrunoRuffo'''"

# To get the tokeniser corresponding to a specific model in the OpenAI API:
# enc = tiktoken.encoding_for_model("gpt-4")

In [22]:
from pprint import pprint
pprint(messages)

[{'content': 'You are an NLP assistant whose purpose is to perform Named '
             'Entity Recognition (NER). NER involves identifying and '
             'classifying named entities in a text into predefined categories '
             'such as person names, organizations, locations, and others. You '
             'will need to use the tags defined below:\n'
             'O means the word doesn’t correspond to any entity.\n'
             'B-PER/I-PER means the word corresponds to the beginning of/is '
             'inside a person entity.\n'
             'B-ORG/I-ORG means the word corresponds to the beginning of/is '
             'inside an organization entity.\n'
             'B-LOC/I-LOC means the word corresponds to the beginning of/is '
             'inside a location entity.\n'
             'Do not try to answer the question! Just tag each token in the '
             'sentence.',
  'role': 'system'},
 {'content': 'Tag the following sentence: "Stratford , Oklahoma"',
  'role': 

## Chat

In [31]:

_ENDPOINT = "https://httpqas26-frontend-qasazap-prod-dsm02p.qas.binginternal.com/completions"
_SCOPES = ["api://68df66a4-cad9-4bfd-872b-c6ddde00d6b2/access"]
_CLIENT_ID = "68df66a4-cad9-4bfd-872b-c6ddde00d6b2"
_AUTHORITY = "https://login.microsoftonline.com/72f988bf-86f1-41af-91ab-2d7cd011db47"
guidance.llm = guidance.llms.SubstrateLLM(auth_method="AAD",
                                        model="gpt-4",
                                        model_name="dev-ppo",
                                        caching=False,
                                        chat_mode="auto",  
                                        client_id=_CLIENT_ID, 
                                        authority=_AUTHORITY, 
                                        endpoint=_ENDPOINT, 
                                        scopes=_SCOPES)


In [32]:
# note that we enclose all of the text in one of the valid role tags for the model
# `system`, `user`, and `assistant` are just shorthand for {{#role name="system"}}...{{/role}}
# the whitepace outside the role tags is ignored by gpt-4, the whitespace inside the role tags is not
# so we use the ~ to remove the whitespace we don't want to give to the model (but want to keep in the code for clarity)
program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
{{conversation_question}}
{{~/user}}

{{! this is a comment. note that we don't have to use a stop="stop_string" for the gen command below because Guidance infers the stop string from the role tag }}
{{#assistant~}}
{{gen 'response'}}
{{~/assistant}}''')



In [33]:
program

Stop program 
 {{#system~}} 
You are a helpful assistant.
 {{~/system}} 

 {{#user~}} 
 {{conversation_question}} 
 {{~/user}} 


 {{#assistant~}} 
 {{gen 'response'}} 
 {{~/assistant}}

In [34]:
executed_program = program(conversation_question='What is the meaning of life?')

Stop program system You are a helpful assistant. user What is the meaning of life? assistant As an AI, I don't have a personal belief about the meaning of life, but this is a question that has puzzled philosophers, theologians, scientists, and many other thinkers for centuries. There are many different answers to this question, and the meaning of life is something that is deeply personal and can vary greatly from person to person. Some people believe that the meaning of life is to seek happiness, to achieve success, to fulfill one's potential, to make the world a better place, to love and be loved, to create and experience beauty, to understand the universe and our place in it, or to connect with something greater than ourselves. Others believe that the meaning of life is determined by individual goals, values, and experiences. Ultimately, the meaning of life is something that each person must determine for themselves based on their own beliefs, values, and experiences.